In [1]:
#DATASET_FILE = "random-xs-20-euclidean.hdf5"
#DATASET_FILE = "random-s-100-euclidean.hdf5"
#DATASET_FILE = "sift-128-euclidean.hdf5"
#DATASET_FILE = "fashion-mnist-784-euclidean.hdf5"
#DATASET_FILE = "mnist-784-euclidean.hdf5"
#DATASET_FILE = "nytimes-16-angular.hdf5"
#DATASET_FILE = "glove-100-angular.hdf5"
#DATASET_FILE = "glove-25-angular.hdf5"
#DATASET_FILE = "stackoverflow-512-angular.hdf5"
DATASET_FILE = "imagenet-96-angular.hdf5"

num_queries = 10
k = 10

In [3]:
import time

## prepare data

In [2]:
import h5py
import numpy as np
from sklearn import preprocessing

dataset = h5py.File(DATASET_FILE, 'r')
        
train_data = np.array(dataset['train'])
d = train_data.shape[1]

test_data = np.array(dataset['test'])[:num_queries]
nn = np.array(dataset['neighbors'])[:num_queries]

if DATASET_FILE.startswith(('glove-', 'stackoverflow-')):
    train_data = preprocessing.normalize(train_data, axis=1, norm='l2')
    test_data = preprocessing.normalize(test_data, axis=1, norm='l2')
    
print('train set of size (%d * %d)' % train_data.shape)
print('test set of size (%d * %d)' % test_data.shape)

train set of size (9990000 * 96)
test set of size (10 * 96)


In [3]:
from sklearn import decomposition
from sklearn import neighbors

PCA_DIMS = 96

np.random.seed(0)
small_data = train_data[0:1000000,]

pca = decomposition.PCA(n_components=PCA_DIMS)
pca.fit(small_data)

cumsum = np.cumsum(pca.explained_variance_ratio_)
effective_dim = sum(cumsum < 0.95)
print("num principle components needed to explain {}% of variance: {}".format(85, effective_dim))

num principle components needed to explain 85% of variance: 76


In [21]:
from sklearn import cluster

small_data=train_data[0:1000,]

kmeans = cluster.KMeans(n_clusters=100, n_init=1, max_iter=10).fit(small_data)

In [22]:
kmeans.inertia_ / len(small_data)

0.2688719017475926

In [39]:
import math
def cosine(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [41]:
np.mean([math.sqrt(np.linalg.norm(kmeans.cluster_centers_[7,] - x)) for x in kmeans.cluster_centers_])

0.8191890743068858

In [93]:
def compute_recall(indices, nn):
    recall = []
    for i in range(0, len(indices)):
        knn = nn[i,:k]
        overlap = np.intersect1d(indices[i], knn)
        recall.append(len(overlap) / k)
    return recall

## kd-forest

In [94]:
import numpy as np
from sklearn import neighbors

kd_forest = neighbors.KDForest(train_data, n_trees=8)

In [95]:
all_indices = []

kd_forest.reset_dist_comps()
start = time.time()
for point in test_data:
    test_point = np.array([point]).astype(np.double)
    _, indices = kd_forest.query(np.array(test_point), k=k, max_dist_comps=500000, tree_index=-1)
    all_indices.append(indices[0])

elapsed = time.time() - start
print("QPS: {:.2f}".format(test_data.shape[0] / elapsed))

recall = compute_recall(all_indices, nn)
print("recall:", np.mean(recall))
    
print("average dist comps: {:.2f}".format(kd_forest.get_dist_comps() / test_data.shape[0]))

QPS: 2.05
recall: 1.0
average dist comps: 267112.90


In [ ]:
import scipy as sp

rotation1 = sp.stats.special_ortho_group.rvs(train_data.shape[1])
rotation2 = sp.stats.special_ortho_group.rvs(train_data.shape[1])

X_rot = np.zeros((2, train_data.shape[1]))
train_data_rot1 = np.dot(train_data, rotation1)
train_data_rot2 = np.dot(train_data, rotation2)

test_point = np.array([test_data[0]])

X_rot[0,:] = np.dot(test_point, rotation1)
X_rot[1,:] = np.dot(test_point, rotation2)

In [ ]:
dist_rot = np.linalg.norm(train_data_rot2 - X_rot[1,:], axis=1)
dist = np.linalg.norm(train_data - test_point, axis=1)

sum(abs(dist_rot - dist)) / train_data.shape[0]
#sum(abs(dist)) / train_data.shape[0]


## sklearn kd-tree

In [54]:
from sklearn import neighbors
kd_tree = neighbors.KDTree(train_data)

In [89]:
start = time.time()

all_indices = []
total_dist_comps = 0
for point in test_data:
    dist, indices = kd_tree.query([point], k=k, breadth_first=True, max_dist_comps=500000)
    all_indices.append(indices[0])
    
    total_dist_comps += kd_tree.get_n_calls() + kd_tree.get_tree_stats()[2]
    kd_tree.reset_n_calls()

elapsed = time.time() - start
print("QPS: {:.2f}".format(test_data.shape[0] / elapsed))

recall = compute_recall(all_indices, nn)
print("recall:", np.mean(recall))
print("average dist comps: {:.2f}".format(total_dist_comps / test_data.shape[0]))

QPS: 5.13
recall: 0.91
average dist comps: 500032.00


In [ ]:
len(all_indices[0])

## sklearn ball tree

In [ ]:
from sklearn import neighbors
np.random.seed(0)
ball_tree = neighbors.BallTree(train_data)

In [ ]:
dist, indices = ball_tree.query(test_data, k=k)
recall = compute_recall(indices, nn)
print(np.mean(recall))

## PCA + kd-tree

In [ ]:
from sklearn import decomposition
from sklearn import neighbors

PCA_DIMS = 32

np.random.seed(0)

pca = decomposition.PCA(n_components=PCA_DIMS)
train_data_pca = pca.fit_transform(train_data)

kd_tree_pca = neighbors.KDTree(train_data_pca)

In [ ]:
start = time.time()

test_data_pca = pca.transform(test_data)

all_indices = []
for i in range(len(test_data)):
    point = test_data[i]
    point_pca = test_data_pca[i]
    
    _, indices = kd_tree_pca.query([point_pca], k=100, breadth_first=True, max_candidates=100000)
    indices = indices[0]
    dists = np.linalg.norm(train_data[indices] - point, axis=1)

    nearest_dists = np.argpartition(dists, k)[:k]
    all_indices.append(indices[nearest_dists])

elapsed = time.time() - start
print("QPS: {:.2f}".format(test_data.shape[0] / elapsed))

recall = compute_recall(all_indices, nn)
print(np.mean(recall))

In [ ]:
dists = np.linalg.norm(train_data[indices] - test_data[0], axis=1)
dists
nearest_dists = np.argpartition(dists, k)[:k]
nearest_dists


In [ ]:
print(test_data_pca.shape)
cumsum = np.cumsum(pca.explained_variance_ratio_)
print("explained variance at {}: {}".format(32, cumsum[32 - 1]))

## random projections + kd-trees

In [ ]:
from sklearn import random_projection

NUM_PROJECTIONS = 8
PROJECTION_DIMS = 8

kd_trees = []
projections = []

for p in range(NUM_PROJECTIONS):
    projection = random_projection.SparseRandomProjection(n_components=PROJECTION_DIMS)
    projections.append(projection)
    
    train_data_proj = projection.fit_transform(train_data)
    kd_tree = neighbors.KDTree(train_data_proj)
    kd_trees.append(kd_tree)

print("num projections: {}".format(len(kd_trees)))

In [ ]:
indices = []

NUM_PROJECTIONS=8

test_data_proj = []
for p in range(NUM_PROJECTIONS):
    projection = projections[p]
    test_data_proj.append(projection.transform(test_data))

start = time.time()
for i in range(test_data.shape[0]):
    candidates = []
    for p in range(NUM_PROJECTIONS):
        point_proj = test_data_proj[p][i]
        
        kd_tree = kd_trees[p]
        _, result = kd_tree.query([point_proj], k=100, breadth_first=True, max_candidates=100000)
        candidates.extend(result[0])

    indices.append(candidates)

elapsed = time.time() - start
print("QPS: {:.2f}".format(test_data.shape[0] / elapsed))

recall = compute_recall(indices, nn)
print("recall:", np.mean(recall))
print(kd_tree.get_tree_stats())

In [ ]:
kd_tree.get_tree_stats()

In [ ]:
indices = []

for i in range(len(test_data)):
    indices.append([])
    for p in range(NUM_PROJECTIONS):
        indices_proj = candidates[p]
        indices[i].extend(indices_proj)

recall = compute_recall(indices, nn)
print(np.mean(recall))

## random rotations + kd-trees

In [ ]:
import scipy as sp

NUM_ROTATIONS = 8

np.random.seed(0)
kd_trees = []
rotations = []

ortho_group = sp.stats.special_ortho_group

for p in range(NUM_ROTATIONS):
    rotation = ortho_group.rvs(train_data.shape[1])
    rotations.append(rotation)

    train_data_rot = np.dot(train_data, rotation)

    kd_tree = neighbors.KDTree(train_data_rot, leaf_size=1000)
    kd_trees.append(kd_tree)

print("num rotations: {}".format(len(kd_trees)))

In [ ]:
indices = []

NUM_ROTATIONS=8

test_data_rot = []
for r in range(NUM_ROTATIONS):
    rotation = rotations[r]
    test_data_rot.append(np.dot(test_data, rotation))

start = time.time()
for i in range(test_data.shape[0]):
    candidates = []
    for r in range(NUM_ROTATIONS):
        point_rot = test_data_rot[r][i]
        
        kd_tree = kd_trees[r]
        _, result = kd_tree.query([point_rot], k=100, breadth_first=False, max_candidates=5000)
        candidates.extend(result[0])

    indices.append(candidates)

elapsed = time.time() - start
print("QPS: {:.2f}".format(test_data.shape[0] / elapsed))

recall = compute_recall(indices, nn)
print("recall:", np.mean(recall))
print(kd_tree.get_tree_stats())

## variance-weighted projections + kd-trees

In [ ]:
from sklearn import decomposition
from sklearn import preprocessing
from sklearn import neighbors

np.random.seed(0)

train_data_var = np.var(train_data, axis=0, keepdims=True)
train_data_var = preprocessing.normalize(train_data_var, axis=1, norm='l1')

In [ ]:
NUM_PROJECTIONS = 4
PROJECTION_DIMS = 32

kd_trees = []
projections = []

for p in range(NUM_PROJECTIONS):
    dims = np.random.choice(d, PROJECTION_DIMS, replace=False, p=train_data_var[0])
    projections.append(dims)
    
    train_data_proj = train_data[:,dims]
    kd_tree = neighbors.KDTree(train_data_proj)
    kd_trees.append(kd_tree)

print("num projections: {}".format(len(kd_trees)))

In [ ]:
candidates = []
start = time.time()

for p in range(NUM_PROJECTIONS):
    dims = projections[p]
    kd_tree = kd_trees[p]
    
    test_data_proj = test_data[:,dims]
    dist, indices = kd_tree.query(test_data_proj, k=100)
    
    candidates.append(indices)
    
elapsed = time.time() - start
print("time: {:.2f} sec".format(elapsed))

In [ ]:
indices = []

for i in range(len(test_data)):
    indices.append([])
    for p in range(NUM_PROJECTIONS):
        indices_proj = candidates[p]
        indices[i].extend(indices_proj)

recall = compute_recall(indices, nn)
print(np.mean(recall))